In [1]:
import numpy as np #We import all our libraries
import matplotlib
import matplotlib.pyplot as plt
from itertools import product
from scipy import optimize # for fits

from sympy import latex, symbols, sin, cos, pi, simplify, lambdify, Matrix

%matplotlib inline
rng = np.random.RandomState(0)
from IPython.display import display, Math
import torch
import torch.autograd.functional as functional

In [2]:
def fgrad(inputs): #This function allow us to create the hessian by calling it later.
    return torch.autograd.functional.jacobian(metric_tensor,inputs, create_graph=True)
def christoffelderiv(coord1, coord2, deriv, alpha, beta): #It's the part of the Riemann tensor with the derivative of the Christoffel.
    christoffelderiv = 0.5*((jacobinv_l[alpha][beta][deriv] * (jacob_l[alpha][coord1][coord2] + jacob_l[alpha][coord2][coord1] - jacob_l[coord1][coord2][alpha]))\
                                    + g_inv_l[alpha][beta] * (hessian_l[alpha][coord1][coord2][deriv] + hessian_l[alpha][coord2][coord1][deriv] - hessian_l[coord1][coord2][alpha][deriv]))
    return christoffelderiv 


## Metric tensor: 2D unit sphere

Coordinates $x^\alpha = (\theta, \phi)$.

\begin{equation}
g_{\mu\nu} = 
\begin{pmatrix}
1 & 0 \\
0 & \sin^2 \theta \\
\end{pmatrix}
\end{equation}

In [3]:
#We define a metric tensor. 
dim=2 #Tell the dimension.

def metric_tensor(coordinate,*arg): #This function create the metric tensor, we need to put in a fonction to be able to
                                    #calculate its derivatives with our jacobian later.
    """
    Here, we have as arg for our function : 
    coordinates : (theta, phi) of a point 
    """
    theta=coordinate[0] 
    phi=coordinate[1]
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1
    tensor[1,1] = torch.pow(torch.sin(theta),2)
    return tensor

def inverse(coordinate, *arg): #Gives us the inverse of the metric_tensor.
    theta=coordinate[0]
    phi=coordinate[1]
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1
    tensor[1,1] = torch.pow(torch.sin(theta),2)
    tensor = torch.inverse(tensor)
    return tensor
def christoffelproduct_liste(coord1, coord2, coord3): #This function gives us the product of two christoffels.
        christoffelproduct = christoffel_l[beta][coord1][eta]*christoffel_l[eta][coord2][coord3]
        return christoffelproduct

In [4]:
#Let's set the initial coordinate to start with.  
coord = torch.tensor([3., 15.],requires_grad=True)
print("theta,phi coordinates: ",coord.data)

theta,phi coordinates:  tensor([ 3., 15.])


## Metric tensor: 4D flat space, spherical coordinates

Coordinates $x^\alpha = (ct, R, \theta, \phi)$.

\begin{equation}
g_{\mu\nu} = 
\begin{pmatrix}
1 & 0 & 0 & 0  \\
0 & -1 & 0 & 0 \\
0 & 0 & -R^2 & 0 \\
0 & 0 & 0 & -R^2 \sin^2 \theta \\
\end{pmatrix}
\end{equation}

In [ ]:
#We define a metric tensor. 
dim=4 #Tell the dimension.

def metric_tensor(coordinate,*arg): 
                                    #This function create the metric tensor, we need to put in a fonction to be able to
                                    #calculate its derivatives with our jacobian later.
    """
    Args: 
        coordinates (ct,r,theta,phi) of a point
        arg: not used
    """
    t=coordinate[0] # c=1
    r=coordinate[1]
    theta=coordinate[2]
    phi=coordinate[3]
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1
    tensor[1,1] = -1
    tensor[2,2] = -1 * torch.pow(r,2)
    tensor[3,3] = -1 * torch.pow(r*torch.sin(theta),2)
    return tensor

def inverse(coordinate, *arg): #And the inverse of the metric tensor. 
    tensor = metric_tensor(coordinate, *arg)
    tensor = torch.inverse(tensor)
    return tensor

In [ ]:
#Let's set the initial coordinate to start with.  
coord = torch.tensor([0, 10, 30, 0.5],requires_grad=True)

## Metric tensor: Schwarzschid metric

Coordinates $x^\alpha = (ct, R, \theta, \phi)$.

$$g_{\mu\nu} = 
\begin{pmatrix}
1 - \frac{2GM}{Rc^2} & 0 & 0 & 0 \\
0 & -\left(1 - \frac{2GM}{Rc^2}\right)^{-1} & 0 & 0 \\
0 & 0 & -R^2 & 0 \\
0 & 0 & 0 & -R^2 \sin^2 \theta \\
\end{pmatrix}
= 
\begin{pmatrix}
1 - \frac{R_s}{R} & 0 & 0 & 0 \\
0 & -\left(1 - \frac{R_s}{R}\right)^{-1} & 0 & 0 \\
0 & 0 & -R^2 & 0 \\
0 & 0 & 0 & -R^2 \sin^2 \theta \\
\end{pmatrix}
$$

With $R_s = \frac{2 G M}{c^2}$, Schwarzschild radius.

In [ ]:
#We define a metric tensor. 
dim=4 #Tell the dimension.
Rs = 1 #Here, we have arguments. Rs is the one argument of this metric tensor.
def metric_tensor(coordinate,*arg):#This function create the metric tensor, we need to put in a fonction to be able to
                                    #calculate its derivatives with our jacobian later.
    """
    Args: 
        coordinates (ct,r,theta,phi) of a point
        arg[0]: Rs, Schwarzschid radius
    """
    t=coordinate[0] # c=1
    r=coordinate[1]
    theta=coordinate[2]
    phi=coordinate[3]
    #if (len(arg)>0):
    #    Rs = arg[0] # Schwarzschid radius
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1 - Rs/r
    tensor[1,1] = -1 * torch.pow(1 - Rs/r,-1)
    tensor[2,2] = -1 * torch.pow(r,2)
    tensor[3,3] = -1 * torch.pow(r*torch.sin(theta),2)
    return tensor

def inverse(coordinate, *arg):
    tensor = metric_tensor(coordinate, *arg)
    tensor = torch.inverse(tensor)
    return tensor

In [ ]:
coord = torch.tensor([1,10,0.5,0.2],requires_grad=True)
print("t,r,theta,phi coordinates: ",coord.data)

In [ ]:
#here, we calculate all that is needed to do the further calculations such as the Christofell and the jacobian.

g = metric_tensor(coord)
g_l = g.tolist() #We also convert it into a list.
g_inv = torch.inverse(g)
g_inv_l = g_inv.tolist()
#We do all the preliminary calculations : 
jacob = functional.jacobian(metric_tensor,(coord)) #We take the jacobian of our metric tensor at the desirated coordinate
jacob_l = jacob.tolist()
jacobinv = functional.jacobian(inverse, (coord)) #And we take alsothe jacobian of the inverse metric tensor as we will need ti.
jacobinv_l = jacobinv.tolist()
hessian = torch.autograd.functional.jacobian(fgrad,(coord), create_graph=True) #fgrad here is the jacobian put in a function. 
#By doing the jacobian of the jacobian of the metric tensor at a given coordinate, one gets the hessian at those coordinates.
hessian_l = hessian.tolist()



In [5]:
#We write down the coordinates
distance = int(input('What distance do you want from the beginning? \n distance = '))
pas = 1 #The step done between each calculations. 
n = distance/pas
n = int(n)
coord_l = coord.tolist() #We convert it to list to better manipulate it. 
liste=[]
for i in range(dim):
    x = np.arange(coord_l[i], coord_l[i]+distance+pas, pas)
    x_l = x.tolist()
    liste.append(x_l)
listetensor = torch.tensor(liste)

#We write down our coordinates. 
f = open("coordinates.txt", "w")
for i in range(dim):
    for j in range(n+1):
        x = str(liste[i][j])
        f.write(x + ',')
    f.write('\n')
f.close() 

f = open("coordinates.txt", "r")
read_list = []
i = 0
for line in f:
    read_list_str = line.split(',')
    read_list_str.pop()
    read_list_scalar = [float(x) for x in read_list_str]
    read_list.append(read_list_scalar)
    print("x_",i,"=", read_list_scalar)
    i=+1
f.close() 



What distance do you want from the beginning? 
 distance = 2
x_ 0 = [3.0, 4.0, 5.0]
x_ 1 = [15.0, 16.0, 17.0]


In [6]:
#Writing new data
actual_list = []
christo_data = open("christo_data.txt", "w") #We create the files for data.
riemann_data = open("riemann_data.txt", "w")
ricci_data = open("ricci_data.txt", "w")
scalarcurvature_data = open("scalarcurvature_data.txt", "w")
for i in range(0,n+1): #What we do there is read the coordinates given before to create all the values corresponding.
    for j in range(0, dim):
        actual_list.append(read_list[j][i])
    coord_ten = torch.tensor(actual_list)
    g = metric_tensor(coord_ten) #Such as the metric tensor
    g_l = g.tolist() 
    g_inv = torch.inverse(g)#Such as the inverse metric tensor
    g_inv_l = g_inv.tolist()
    #We do all the preliminary calculations : 
    jacob = functional.jacobian(metric_tensor,(coord_ten)) #We take the jacobian of our metric tensor at the desirated coordinate
    jacob_l = jacob.tolist() 
    jacobinv = functional.jacobian(inverse, (coord_ten))
    jacobinv_l = jacobinv.tolist()
    hessian = torch.autograd.functional.jacobian(fgrad,(coord_ten), create_graph=True)
    hessian_l = hessian.tolist()
    
    #Christoffel
    christoffel = torch.zeros(dim,dim,dim)
    for beta in range(0,dim):
        for mu in range(0,dim):
            for nu in range(0,dim):
                cricri = 0
                for alpha in range(0,dim): #Here the loop are done in respect the einstein summation convention.

                    cricri += 0.5 * g_inv_l[alpha][beta] * (jacob_l[alpha][mu][nu] + jacob_l[alpha][nu][mu] - jacob_l[mu][nu][alpha])
                christoffel[beta,mu,nu] = cricri
            
    christoffel_l = christoffel.tolist() #After having written down a list, we need to write down the data in our file.
    for beta in range(0,dim):
        for mu in range(0,dim):
            for nu in range(0,dim):
                x = str(christoffel_l[beta][mu][nu])
                christo_data.write(x + ',')#We follow a simple 
            christo_data.write('_')
        christo_data.write(':')
    christo_data.write('\n')
#We follow a simple indice code to separate our data :  
    #coord : "/n"                               
        #beta : ":"
            #epsilon : "/"
                # mu : "_"
                    # nu : ","
    #Riemann
    riemann = torch.zeros(dim,dim,dim,dim)
    riemann_l = riemann.tolist()
    for beta in range(0,dim):
        for epsilon in range(0,dim):
            for mu in range(0,dim):
                for nu in range(0,dim):
                    riemannscalar = 0
                    for alpha in range(0,dim): 
                        #Here as you can see, the riemann tensor is divided in the addition of those two functions
                        riemannscalar += christoffelderiv(mu, epsilon, nu, alpha, beta)-christoffelderiv(mu, nu, epsilon, alpha, beta)
                        #First the christoffel derivatives
                    for eta in range(0,dim):
                        #Then the christoffelproduct.
                        riemannscalar += christoffelproduct_liste(nu, mu, epsilon) - christoffelproduct_liste(epsilon, mu, nu) 
                    riemann_l[beta][mu][nu][epsilon] = riemannscalar
    #We write down the data for the riemann tensor.
    for beta in range(0,dim):
        for epsilon in range(0,dim):
            for mu in range(0,dim):
                for nu in range(0,dim):
                    x = str(riemann_l[beta][mu][nu][epsilon])
                    riemann_data.write(x + ',')
                riemann_data.write('_')
            riemann_data.write('/')
        riemann_data.write(':')
    riemann_data.write('\n')
    
    #Ricci 
    ricci = torch.zeros(dim,dim)
    ricci_l = ricci.tolist()
    for mu in range(0, dim):
        for nu in range(0, dim):
            riccisca = 0
            for alpha in range(0, dim):
                riccisca += riemann_l[alpha][mu][alpha][nu]
                ricci_l[mu][nu]= riccisca
                
    #We write down the data for the ricci tensor.
    for mu in range(0,dim):
        for nu in range(0,dim):
            x = str(ricci_l[mu][nu])
            print(x)
            ricci_data.write(x + ',')
        ricci_data.write('_')
    ricci_data.write('\n')
    
    #Scalar curvature
    scalarcurvature = 0
    for mu in range(dim):
        for nu in range(dim):
            scalarcurvature += g_inv_l[mu][nu]*ricci_l[mu][nu]  
    x = str(scalarcurvature)
    scalarcurvature_data.write(x + '\n')
    
    actual_list.pop()
    actual_list.pop()
#We close our files.
christo_data.close() 
riemann_data.close() 
ricci_data.close()
scalarcurvature_data.close() 

1.0000024587332064
0.0
0.0
0.019914978040077358
1.0000000048060222
0.0
0.0
0.5727500452269982
1.0000000042101966
0.0
0.0
0.9195358071473692


In [14]:
## Reading data 
#We follow a simple indice code to separate our data :  
    #coord : "/n"                               
        #beta : ":"
            #epsilon : "/"
                # mu : "_"
                    # nu : ","
#Using it, we will be able to put anew in list our data, as it was when we wrote it down.
def reader(): 
    choice = input("What do you want to read ? Choose a number : \n 1 : Christoffel_symbols \n 2 : Riemann_tensor \n 3 : Ricci_tensor \n 4 : Scalar_curvature ")
    read_list = []
    if choice =='1': 
        f = open("christo_data.txt", "r")
        #Christo
        for line in f:
            read_list_str = line.split(':')
            read_list_str.pop()
            for i in range(len(read_list_str)):
                read_list_str[i] = read_list_str[i].split('_')
            for i in read_list_str : #Here, i use a loop to suppress all the '' that are relics of the conversion.
                for j in i: 
                    if j == '': 
                        i.remove(j)
                for j in range(len(i)):
                    i[j] =i[j].split(',')
                for j in i:
                    j.pop()
                    read_list_scalar = [float(x) for x in j]
                    read_list.append(read_list_scalar)
            print(read_list)
    elif choice =='2':
        f = open("riemann_data.txt", "r")
        #Riemann
        for line in f:
            read_list_str = line.split(':')
            read_list_str.pop()
            for i in range(len(read_list_str)):
                read_list_str[i] = read_list_str[i].split('/')
            for i in read_list_str : #Here, i use a loop to suppress all the '' that are relics of the conversion.
                for j in i: 
                    if j == '': 
                        i.remove(j)
                for j in range(len(i)):
                    i[j] =i[j].split('_')
                for j in i:
                    for h in range(len(j)):
                        j[h] =j[h].split(',')
                    j.pop()
                    for h in j: 
                        h.pop()
                        read_list_scalar = [float(x) for x in h]
                        read_list.append(read_list_scalar)
            print(read_list)
    elif choice =='3':
        f = open("ricci_data.txt", "r")
        #Ricci
        for line in f:
            read_list_str = line.split('_')
            read_list_str.pop()
            for i in range(len(read_list_str)):
                read_list_str[i] = read_list_str[i].split(',')
            for i in read_list_str : #Here, i use a loop to suppress all the '' that are relics of the conversion.
                for j in i: 
                    if j == '':
                        i.remove(j)
                read_list_scalar = [float(x) for x in i]
                read_list.append(read_list_scalar)
        print(read_list)
    elif choice == '4':
        f = open("scalarcurvature_data.txt", "r")
        #scalar curvature
        read_list = [float(x) for x in f]
        print(read_list)
    else : 
        print("That's not a number, try again :)")
    f.close() 
reader()

What do you want to read ? Choose a number : 
 1 : Christoffel_symbols 
 2 : Riemann_tensor 
 3 : Ricci_tensor 
 4 : Scalar_curvature 1
[[0.0, 0.0], [0.0, 0.1397077441215515], [0.0, -7.01525354385376], [-7.01525354385376, 0.0]]
[[0.0, 0.0], [0.0, 0.1397077441215515], [0.0, -7.01525354385376], [-7.01525354385376, 0.0], [0.0, 0.0], [0.0, -0.49467912316322327], [0.0, 0.8636911511421204], [0.8636911511421204, 0.0]]
[[0.0, 0.0], [0.0, 0.1397077441215515], [0.0, -7.01525354385376], [-7.01525354385376, 0.0], [0.0, 0.0], [0.0, -0.49467912316322327], [0.0, 0.8636911511421204], [0.8636911511421204, 0.0], [0.0, 0.0], [0.0, 0.27201056480407715], [0.0, -0.2958129048347473], [-0.2958129048347473, 0.0]]


In [38]:
def Relat_calc_files():
    distance = 10
    pas = 1
    n = distance/pas
    n = int(n)
    coord_l = coord.tolist()
    liste=[]
    for i in range(dim):
        x = np.arange(coord_l[i], coord_l[i]+distance+pas, pas)
        x_l = x.tolist()
        liste.append(x_l)
    listetensor = torch.tensor(liste)
    
    #We write down our coordinates. 
    f = open("coordinates.txt", "w")
    for i in range(dim):
        for j in range(n+1):
            x = str(liste[i][j])
            f.write(x + ',')
        f.write('\n')
    f.close() 

    f = open("coordinates.txt", "r")

    for line in f:
        read_list_str = line.split(',')
        read_list_str.pop()
        read_list = [float(x) for x in read_list_str]
        print(read_list)
    f.close() 


    def christoffelproduct_liste(coord1, coord2, coord3):
        christoffelproduct = christoffel_l[beta][coord1][eta]*christoffel_l[eta][coord2][coord3]
        return christoffelproduct
    # Initialize Christoffel
    christoffel = torch.zeros(dim,dim,dim)
    for beta in range(0,dim):
        for mu in range(0,dim):
            for nu in range(0,dim):
                cricri = 0
                for alpha in range(0,dim):

                    cricri += 0.5 * g_inv_l[alpha][beta] * (jacob_l[alpha][mu][nu] + jacob_l[alpha][nu][mu] - jacob_l[mu][nu][alpha])
                christoffel[beta,mu,nu] = cricri
            
    christoffel_l = christoffel.tolist()
    christo_data = open("christo_data.txt", "w")
    for beta in range(0,dim):
        for mu in range(0,dim):
            for nu in range(0,dim):
                x = str(christoffel_l[beta][mu][nu])
                christo_data.write(x + ',')
            christo_data.write(':')
        christo_data.write('\n')
    christo_data.close() 
    # Initialize Riemannian tensor
    riemann = torch.zeros(dim,dim,dim,dim)
    riemann_l = riemann.tolist()
    for beta in range(0,dim):
        for epsilon in range(0,dim):
            for mu in range(0,dim):
                for nu in range(0,dim):
                    riemannscalar = 0
                    for alpha in range(0,dim):
                        riemannscalar += christoffelderiv(mu, epsilon, nu, alpha, beta)-christoffelderiv(mu, nu, epsilon, alpha, beta)
                    for eta in range(0,dim):
                        riemannscalar += christoffelproduct_liste(nu, mu, epsilon) - christoffelproduct_liste(epsilon, mu, nu) 
                    riemann_l[beta][mu][nu][epsilon] = riemannscalar

    riemann_data = open("riemann_data.txt", "w")
    for beta in range(0,dim):
        for epsilon in range(0,dim):
            for mu in range(0,dim):
                for nu in range(0,dim):
                    x = str(riemann_l[beta][mu][nu][epsilon])
                    riemann_data.write(x + ',')
                riemann_data.write(':')
            riemann_data.write('/')
        riemann_data.write('\n')
    riemann_data.close() 
    
    
    
    #Initialize Ricci tensor
    ricci = torch.zeros(dim,dim)
    ricci_l = ricci.tolist()
    for mu in range(0, dim):
        for nu in range(0, dim):
            riccisca = 0
            for alpha in range(0, dim):
                riccisca += riemann_l[alpha][mu][alpha][nu]
                ricci_l[mu][nu]= riccisca
                
    ricci_data = open("ricci_data.txt", "w")
    for mu in range(0,dim):
        for nu in range(0,dim):
            x = str(ricci_l[mu][nu])
            ricci_data.write(x + ',')
        ricci_data.write('\n')
    ricci_data.close() 
    
    
    
    #scalarcurvature 
    scalarcurvature = 0
    for mu in range(dim):
        for nu in range(dim):
            scalarcurvature += g_inv_l[mu][nu]*ricci_l[mu][nu]  
            
    scalarcurvature_data = open("scalarcurvature_data.txt", "w")
    x = str(scalarcurvature)
    scalarcurvature_data.write(x + ',')
    scalarcurvature_data.close() 
    
    
    
    
    i = 0
    while i < 1:
        print("What do you want to display ? Choose a number : \n 1 : Christoffel_symbols \n 2 : Riemann_tensor \n 3 : Ricci_tensor \n 4 : Scalar_curvature ")
        detail = input()
        if detail== '1':
            for beta in range(0,dim):
                for mu in range(0,dim):
                    for nu in range(0,dim):
                        display(Math(f'\Gamma^{{{beta}}}_{{{mu} {nu}}} = ' + latex(christoffel_l[beta][mu][nu])))
        elif detail=='2':
            for beta in range(0,dim):
                for epsilon in range(0,dim):
                    for mu in range(0,dim):
                        for nu in range(0,dim):
                            display(Math(f'R^{{{beta}}}_{{{mu} {nu} {epsilon}}} = ' + latex(riemann_l[beta][mu][nu][epsilon])))
        elif detail=='3':
            for mu in range(0, dim):
                for nu in range(0, dim):
                    display(Math(f'R_{{{mu} {nu}}} = ' + latex(ricci_l[mu][nu])))
        elif detail=='4':
            display(Math(f'R = ' + latex(scalarcurvature)))
        else :
            print("Error : please choose a number between 1, 2, 3 and 4.")
        print("Do you want to display an other value ? y/n")
        x = input()
        if x == 'n':
            i += 1
Relat_calc_files()


[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0]
[15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0]
What do you want to display ? Choose a number : 
 1 : Christoffel_symbols 
 2 : Riemann_tensor 
 3 : Ricci_tensor 
 4 : Scalar_curvature 
2


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Do you want to display an other value ? y/n


KeyboardInterrupt: Interrupted by user